# Imports and settings

In [139]:
#from elasticsearch_dsl import Search, Q, Range, connections
#from elasticsearch.client import Elasticsearch
from tqdm.notebook import tqdm
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
import re
from openai import OpenAI
import numpy as np

In [140]:
DATA_FOLDER = "../data/events/browse_ai/" 

In [141]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 20)
pd.set_option('display.max_colwidth', 300)

In [142]:
df = pd.read_pickle(f"{DATA_FOLDER}df_browse_ai before description 2024-12-19_18-27-21.pkl")

In [143]:
len(df)

1758

In [144]:
df.replace("nan", np.nan, inplace=True)

/tmp/ipykernel_4543/1857460764.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace("nan", np.nan, inplace=True)


In [145]:
extracted_columns = [col for col in df.columns if col.startswith("extr")]
required_cols = ["Event_URL", "isOnMultipleDays"]
required_cols.extend(extracted_columns)
required_cols_except_start_date = required_cols.copy()
required_cols_except_start_date.remove("extrStartDate")

In [146]:
df['count_notna'] = df[required_cols].notna().sum(axis=1)

In [147]:
df = df.loc[df.groupby('Event_URL')['count_notna'].idxmax()]

In [148]:
len(df)

518

### manual data replacements

In [149]:
flt = df["Event_description"].str.contains("Caspar David Friedrich. Wo alles begann", na=False)
id = max(df.index) + 1
print(id)
tmp = df[flt]
assert len(df[flt])>0 
df = df[~flt]
tmp = tmp.iloc[0]
print(tmp)
df.loc[id]=tmp

df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-31"
df.loc[id, "extrCity"]="Dresden"
df.loc[id, "extrStreetHouseNumber"]="Tzschirnerpl. 2"
df.loc[id, "extrMultipleDaysDetails"] = """Freitag-Montag, Mittwoch 10-18 Uhr Donnerstag 10-21 Uhr Dienstag geschlossen"""



45188
Date_(GMT)                                                                                                                                          2024-09-17 13:34:31
Job_Link                              https://dashboard.browse.ai/teams/meslis/robots/7afcec8c-6cf5-48ab-8bfe-561611c64908/history/f8672f5c-0855-4bc7-804b-77f454bfbc07
Origin_URL                                                                                                               https://sachsen-net.com/veranstaltungen/heute/
Event_URL                                                            https://sachsen-net.com/veranstaltungen/heute/Caspar-David-Friedrich-Der-Maler-Dresden-118476.html
Event_name                                                                                                                           Caspar David Friedrich - Der Maler
                                                                                                    ...                                                   

In [150]:
flt = df["Event_description"].str.contains("Napoleon mit seiner Grande Armée marschierte am 24.", na=False)
id = max(df.index) + 1
print(id)
tmp = df[flt]
assert len(df[flt])>0 
df = df[~flt]
tmp = tmp.iloc[0]
print(tmp)
df.loc[id]=tmp
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-27"
df.loc[id, "extrEndDate"] = "2024-10-31"
df.loc[id, "extrCity"]="Dresden"
df.loc[id, "extrLocationName"]="Militärhistorisches Museum der Bundeswehr"
df.loc[id, "extrStreetHouseNumber"]="Olbrichtpl. 2"
df.loc[id, "extrLocationDetails"]="Olbrichtpl. 2, 01099 Dresden"
df.loc[id, "extrMultipleDaysDetails"] = """Montag 10 - 21 Uhr Dienstag, Donnerstag, Freitag, Samstag, Sonntag 10-18 Uhr Mittwoch geschlossen"""


45189
Date_(GMT)                                                                                                                                          2024-09-27 13:36:19
Job_Link                              https://dashboard.browse.ai/teams/meslis/robots/7afcec8c-6cf5-48ab-8bfe-561611c64908/history/54b0a273-9eb4-4104-8684-805333496372
Origin_URL                                                                                                               https://sachsen-net.com/veranstaltungen/heute/
Event_URL                                     https://sachsen-net.com/veranstaltungen/heute/Sonderausstellung-Napoleon-Muss-Untergehen-Barclay-De-Tolly-Mit-118320.html
Event_name                                                                          Sonderausstellung »Napoleon muss untergehen« Barclay de Tolly mit Feder und Schwert
                                                                                                    ...                                                   

In [151]:
flt = df["Event_description"].str.contains("Wer entscheidet eigentlich, wer dazugehört", na=False)
id = max(df.index) + 1
print(id)
tmp = df[flt]
assert len(df[flt])>0 
df = df[~flt]
tmp = tmp.iloc[0]
print(tmp)
df.loc[id]=tmp
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-31"
df.loc[id, "extrCity"]="Dresden"
df.loc[id, "extrLocationName"]="Robotron-Kantine in Dresden"
df.loc[id, "extrStreetHouseNumber"]="Zinzendorfstraße 5"
df.loc[id, "extrLocationDetails"]="Zinzendorfstraße 5, 01069 Dresden"
df.loc[id, "extrMultipleDaysDetails"] = """Mi – Fr 16 – 19 Uhr Sa & So und Feiertag 12 -18 Uhr"""


45190
Date_(GMT)                                                                                                                                            2024-09-09 13:33:49
Job_Link                                                                                                                             716e2ebb-ecc3-400f-8065-1e0ab9d15cb1
Origin_URL                                                                                                                 https://sachsen-net.com/veranstaltungen/heute/
Event_URL                             https://sachsen-net.com/veranstaltungen/heute/Dazugehoren-Belonging-Zeitgenoessische-Kunst-Performances-Und-Konzerte-Zu-118418.html
Event_name                                                         DAZUGEHÖREN! BELONGING! Zeitgenössische Kunst, Performances und Konzerte zu Zusammenhalt und Verortung
                                                                                                     ...                                        

In [152]:
flt = df["Event_description"].str.contains("Eine Ausstellung zeitgenössischer Kunst zu den Auswirkungen rassistischer", na=False)
id = max(df.index) + 1
print(id)
tmp = df[flt]
assert len(df[flt])>0 
df = df[~flt]
tmp = tmp.iloc[0]
print(tmp)
df.loc[id]=tmp
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-31"
df.loc[id, "extrCity"]="Dresden"
df.loc[id, "extrLocationName"]="Robotron-Kantine in Dresden"
df.loc[id, "extrStreetHouseNumber"]="Zinzendorfstraße 5"
df.loc[id, "extrLocationDetails"]="Zinzendorfstraße 5, 01069 Dresden"
df.loc[id, "extrMultipleDaysDetails"] = """Mi – Fr 16 – 19 Uhr Sa & So und Feiertag 12 -18 Uhr"""


45191
Date_(GMT)                                                                                                                                          2024-09-12 13:36:10
Job_Link                              https://dashboard.browse.ai/teams/meslis/robots/7afcec8c-6cf5-48ab-8bfe-561611c64908/history/a0d0e5aa-5b94-421e-8003-303e051bba3c
Origin_URL                                                                                                               https://sachsen-net.com/veranstaltungen/heute/
Event_URL                                                 https://sachsen-net.com/veranstaltungen/heute/Offener-Prozess-Ausstellung-Zum-Nsu-Komplex-Dresden-118419.html
Event_name                                                                                                                 OFFENER PROZESS. Ausstellung zum NSU-Komplex
                                                                                                    ...                                                   

In [153]:
flt = df["Event_description"].str.contains('Unter dem Motto "Über die Natur" findet zum vierten Mal ', na=False)
id = max(df.index) + 1
print(id)
tmp = df[flt]
assert len(df[flt])>0 
df = df[~flt]
tmp = tmp.iloc[0]
print(tmp)
df.loc[id]=tmp
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-31"
df.loc[id, "extrCity"]="Dresden"
df.loc[id, "extrLocationName"]="Schloss Pillnitz"
df.loc[id, "extrLocationDetails"]="August-Böckstiegel-Straße 2, 01326 Dresden"
df.loc[id, "extrStreetHouseNumber"]="August-Böckstiegel-Straße 2"
df.loc[id, "extrMultipleDaysDetails"] = """täglich 10-17 Uhr, Montag geschlossen"""


45192
Date_(GMT)                                                                                                                                          2024-09-13 13:36:06
Job_Link                              https://dashboard.browse.ai/teams/meslis/robots/7afcec8c-6cf5-48ab-8bfe-561611c64908/history/3450fc8f-0096-46d4-a7e3-0353b1b7451c
Origin_URL                                                                                                               https://sachsen-net.com/veranstaltungen/heute/
Event_URL                                    https://sachsen-net.com/veranstaltungen/heute/Artists-Conquest-Kunst-Erobert-Schloss-Pillnitz-Dresden-Pillnitz-117972.html
Event_name                                                                                                         »Artists' Conquest« - Kunst erobert Schloss Pillnitz
                                                                                                    ...                                                   

In [154]:
flt = df["Event_description"].str.contains('Mikroschnitzereien auf Streichhölzern oder in Walnussschalen, Modelle und Puppenspielzeug', na=False)
id = max(df.index) + 1
print(id)
tmp = df[flt]
assert len(df[flt])>0 
df = df[~flt]
tmp = tmp.iloc[0]
print(tmp)
df.loc[id]=tmp
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-20"
df.loc[id, "extrCity"]="Dresden"
df.loc[id, "extrLocationDetails"]="Köpckestraße 1, 01097 Dresden"
df.loc[id, "extrStreetHouseNumber"]="Köpckestraße 1"
df.loc[id, "extrMultipleDaysDetails"] = """täglich 10-18 Uhr, Montag geschlossen"""


45193
Date_(GMT)                                                                                                                                          2024-09-11 13:35:10
Job_Link                              https://dashboard.browse.ai/teams/meslis/robots/7afcec8c-6cf5-48ab-8bfe-561611c64908/history/19403c5b-c176-4d67-bc6c-b92726ccf34c
Origin_URL                                                                                                               https://sachsen-net.com/veranstaltungen/heute/
Event_URL                                        https://sachsen-net.com/veranstaltungen/heute/Miniminiminiaturen-Die-Kleinste-Ausstellung-Der-Welt-Dresden-115290.html
Event_name                                                                                                       Miniminiminiaturen - Die kleinste Ausstellung der Welt
                                                                                                    ...                                                   

In [155]:
flt = df["Event_description"].str.contains('Dem langjährigen Professor für Monumentalmalerei widmet sich die neue Sonderausstellung', na=False)
id = max(df.index) + 1
print(id)
tmp = df[flt]
assert len(df[flt])>0 
df = df[~flt]
tmp = tmp.iloc[0]
print(tmp)
df.loc[id]=tmp
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-31"
df.loc[id, "extrCity"]="Dresden"
df.loc[id, "extrLocationName"]="Schloss Pillnitz"
df.loc[id, "extrLocationDetails"]="August-Böckstiegel-Straße 2, 01326 Dresden"
df.loc[id, "extrStreetHouseNumber"]="August-Böckstiegel-Straße 2"
df.loc[id, "extrMultipleDaysDetails"] = """Dienstag bis Sonntag 10 bis 17 Uhr"""


45194
Date_(GMT)                                                                                                                                          2024-09-13 13:36:06
Job_Link                              https://dashboard.browse.ai/teams/meslis/robots/7afcec8c-6cf5-48ab-8bfe-561611c64908/history/3450fc8f-0096-46d4-a7e3-0353b1b7451c
Origin_URL                                                                                                               https://sachsen-net.com/veranstaltungen/heute/
Event_URL                                   https://sachsen-net.com/veranstaltungen/heute/Sonderausstellung-Monumental-Der-Maler-Ludwig-Von-Hofmann-Dresden-116301.html
Event_name                                                                                               Sonderausstellung »Monumental!« – Der Maler Ludwig von Hofmann
                                                                                                    ...                                                   

In [156]:
flt = df["Event_description"].str.contains('Tonband- und Kassettengeräte kam man', na=False)
id = max(df.index) + 1
print(id)
tmp = df[flt]
assert len(df[flt])>0 
df = df[~flt]
tmp = tmp.iloc[0]
print(tmp)
df.loc[id]=tmp
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-19"
df.loc[id, "extrEndDate"] = "2024-10-31"
df.loc[id, "extrCity"]="Dresden"
df.loc[id, "extrLocationName"]=" Städtische Galerie Dresden"
df.loc[id, "extrMultipleDaysDetails"] = """Eröffnung: DO 19.09. 19.00 Uhr DI bis SO 10.00 bis 18.00 Uhr FR 10.00 bis 19.00 Uhr"""


45195
Date_(GMT)                                                                                                                                          2024-09-20 13:35:51
Job_Link                              https://dashboard.browse.ai/teams/meslis/robots/7afcec8c-6cf5-48ab-8bfe-561611c64908/history/dc855d8e-2bee-4eb5-ab2a-0155368028a8
Origin_URL                                                                                                               https://sachsen-net.com/veranstaltungen/heute/
Event_URL                                                   https://sachsen-net.com/veranstaltungen/heute/Sonderausstellung-H-G-Griese-Fast-Forward-Dresden-118205.html
Event_name                                                                                                              Sonderausstellung "H. G. Griese – Fast Forward"
                                                                                                    ...                                                   

In [157]:
flt = df["Event_description"].str.contains('100 Jahre KUNSTAUSSTELLUNG KÜHL in Dresden.', na=False)
id = max(df.index) + 1
print(id)
tmp = df[flt]
assert len(df[flt])>0 
df = df[~flt]
tmp = tmp.iloc[0]
print(tmp)
df.loc[id]=tmp
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-31"
df.loc[id, "extrCity"]="Dresden"
df.loc[id, "extrLocationName"]=" Kunstausstellung Kühl Dresden"
df.loc[id, "extrStreetHouseNumber"]="Nordstraße 5"
df.loc[id, "extrMultipleDaysDetails"] = """Mi bis Fr 11 –19 + Sa 11 –16 Uhr"""


45196
Date_(GMT)                                                                                                                                          2024-09-10 13:37:27
Job_Link                              https://dashboard.browse.ai/teams/meslis/robots/7afcec8c-6cf5-48ab-8bfe-561611c64908/history/0758270f-255b-4652-9301-72e97cb58f67
Origin_URL                                                                                                               https://sachsen-net.com/veranstaltungen/heute/
Event_URL                                       https://sachsen-net.com/veranstaltungen/heute/Exposition-100-Jahre-Kunstausstellung-Kuhl-In-Dresden-Dresden-118695.html
Event_name                                                                                                      Exposition: 100 Jahre KUNSTAUSSTELLUNG KÜHL in Dresden.
                                                                                                    ...                                                   

In [158]:
flt = df["Event_description"].str.contains('Auf die Elbe, fertig, los!', na=False)
id = max(df.index) + 1
print(id)
tmp = df[flt]
assert len(df[flt])>0 
df = df[~flt]
tmp = tmp.iloc[0]
print(tmp)
df.loc[id]=tmp
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-31"
df.loc[id, "extrCity"]="Dresden"
df.loc[id, "extrLocationName"]="Blaues Wunder"
df.loc[id, "extrStartTime"] = "11"
df.loc[id, "extrEndTime"] = "15"

45197
Date_(GMT)                                                                                                                                          2024-09-17 13:34:31
Job_Link                              https://dashboard.browse.ai/teams/meslis/robots/7afcec8c-6cf5-48ab-8bfe-561611c64908/history/f8672f5c-0855-4bc7-804b-77f454bfbc07
Origin_URL                                                                                                               https://sachsen-net.com/veranstaltungen/heute/
Event_URL                                                              https://sachsen-net.com/veranstaltungen/heute/Schlauchboottour-Durch-Dresden-Dresden-114641.html
Event_name                                                                                                                               SchlauchbootTour durch Dresden
                                                                                                    ...                                                   

In [159]:
flt = df["Event_description"].str.contains('Das Highlight für Kinder ist die Parkeisenbahn.', na=False)
id = max(df.index) + 1
print(id)
tmp = df[flt]
assert len(df[flt])>0 
df = df[~flt]
tmp = tmp.iloc[0]
print(tmp)
df.loc[id]=tmp
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-26"
df.loc[id, "extrCity"]="Dresden"
df.loc[id, "extrLocationName"]="Parkeisenbahn Dresden"
df.loc[id, "extrLocationDetails"]="01219 Dresden-Altstadt"
df.loc[id, "extrMultipleDaysDetails"] = """Mittwoch bis Freitag: 13:00 bis 18:00 Uhr Samstag, Sonntag, Feiertag: 10:30 bis 18:00 Uhr"""

45198
Date_(GMT)                                                                                                                                          2024-09-10 13:37:27
Job_Link                              https://dashboard.browse.ai/teams/meslis/robots/7afcec8c-6cf5-48ab-8bfe-561611c64908/history/0758270f-255b-4652-9301-72e97cb58f67
Origin_URL                                                                                                               https://sachsen-net.com/veranstaltungen/heute/
Event_URL                                             https://sachsen-net.com/veranstaltungen/heute/Mit-Volldampf-Durch-Den-Park-Die-Dresdner-Parkeisenbahn-115315.html
Event_name                                                                                                     Mit Volldampf durch den Park: Die Dresdner Parkeisenbahn
                                                                                                    ...                                                   

In [160]:
flt = df["Event_description"].str.contains('Süße, halb tote Katze verschwunden! Ihre Spur', na=False)
id = max(df.index) + 1
print(id)
tmp = df[flt]
assert len(df[flt])>0 
df = df[~flt]
tmp = tmp.iloc[0]
print(tmp)
df.loc[id]=tmp
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-31"
df.loc[id, "extrCity"]="Dresden"
df.loc[id, "extrLocationName"]="Technische Sammlungen Dresden"
df.loc[id, "extrLocationDetails"]="01277 Dresden Junghannsstraße 1-3"
df.loc[id, "extrStreetHouseNumber"]="Junghannsstraße 1-3"
df.loc[id, "extrMultipleDaysDetails"] = """DI bis FR 9.00 bis 17.00 Uhr SA/SO 10.00 bis 18.00 Uhr"""

45199
Date_(GMT)                                                                                                                                          2024-09-10 13:37:27
Job_Link                              https://dashboard.browse.ai/teams/meslis/robots/7afcec8c-6cf5-48ab-8bfe-561611c64908/history/0758270f-255b-4652-9301-72e97cb58f67
Origin_URL                                                                                                               https://sachsen-net.com/veranstaltungen/heute/
Event_URL                                  https://sachsen-net.com/veranstaltungen/heute/Escape-Room-Katze-Q-Neuer-Interaktiver-Ausstellungsbereich-Dresden-115568.html
Event_name                                                                                                 Escape Room Katze Q - Neuer interaktiver Ausstellungsbereich
                                                                                                    ...                                                   

In [161]:
flt = df["Event_description"].str.contains('prägenden Periode in der Geschichte des DHMD und stellt sie in den Kontext der wohl einschneidendsten Umbrüche der', na=False)
id = max(df.index) + 1
print(id)
tmp = df[flt]
assert len(df[flt])>0 
df = df[~flt]
tmp = tmp.iloc[0]
print(tmp)
df.loc[id]=tmp
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-31"
df.loc[id, "extrCity"]="Dresden"
df.loc[id, "extrLocationName"]="Hygienemuseum"
df.loc[id, "extrLocationDetails"]="01069 Dresden Lingnerplatz 1"
df.loc[id, "extrStreetHouseNumber"]="Lingnerplatz 1"
df.loc[id, "extrMultipleDaysDetails"] = """Dienstag bis Sonntag, Feiertage: 10 bis 18 Uhr"""

45200
Date_(GMT)                                                                                                                                          2024-09-10 13:37:27
Job_Link                              https://dashboard.browse.ai/teams/meslis/robots/7afcec8c-6cf5-48ab-8bfe-561611c64908/history/0758270f-255b-4652-9301-72e97cb58f67
Origin_URL                                                                                                               https://sachsen-net.com/veranstaltungen/heute/
Event_URL                                                       https://sachsen-net.com/veranstaltungen/heute/Veb-Museum-Das-Deutsche-Hygiene-Museum-In-Der-115656.html
Event_name                                                                                                          VEB MUSEUM - Das Deutsche Hygiene-Museum in der DDR
                                                                                                    ...                                                   

In [162]:
flt = df["Event_URL"].str.contains('https://sachsen-net.com/veranstaltungen/heute/Kinderbiennale-Planet-Utopia-Dresden-114311.html', na=False)
id = max(df.index) + 1
print(id)
tmp = df[flt]
assert len(df[flt])>0 
df = df[~flt]
tmp = tmp.iloc[0]
print(tmp)
df.loc[id]=tmp
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-31"
df.loc[id, "extrCity"]="Dresden"
df.loc[id, "extrLocationDetails"]="01097 Dresden Palaisplatz 11"
df.loc[id, "extrStreetHouseNumber"]="Palaisplatz 11"
df.loc[id, "extrMultipleDaysDetails"] = "Diensatg bis Sonntag 10 bis 18 Uhr"

45201
Date_(GMT)                                                                                                                                          2024-09-10 13:37:27
Job_Link                              https://dashboard.browse.ai/teams/meslis/robots/7afcec8c-6cf5-48ab-8bfe-561611c64908/history/0758270f-255b-4652-9301-72e97cb58f67
Origin_URL                                                                                                               https://sachsen-net.com/veranstaltungen/heute/
Event_URL                                                                https://sachsen-net.com/veranstaltungen/heute/Kinderbiennale-Planet-Utopia-Dresden-114311.html
Event_name                                                                                                                               Kinderbiennale "Planet Utopia"
                                                                                                    ...                                                   

In [163]:
flt = df["Event_URL"].str.contains('https://sachsen-net.com/veranstaltungen/heute/Wiederentdeckt-Dresdner-Malerinnen-Der-Romantik-Dresden-116639.html', na=False)
id = max(df.index) + 1
print(id)
tmp = df[flt]
assert len(df[flt])>0 
df = df[~flt]
tmp = tmp.iloc[0]
print(tmp)
df.loc[id]=tmp
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-31"
df.loc[id, "extrCity"]="Dresden"
df.loc[id, "extrLocationDetails"]="01097 Dresden Hauptstr. 13"
df.loc[id, "extrMultipleDaysDetails"] = "MI bis FR 10.00 bis 17.00 Uhr SA/SO 12.00 bis 17.00 Uhr"

45202
Date_(GMT)                                                                                                                                          2024-09-19 13:35:56
Job_Link                              https://dashboard.browse.ai/teams/meslis/robots/7afcec8c-6cf5-48ab-8bfe-561611c64908/history/acf9cd27-f744-4a25-987d-7f0c3dc30e58
Origin_URL                                                                                                               https://sachsen-net.com/veranstaltungen/heute/
Event_URL                                             https://sachsen-net.com/veranstaltungen/heute/Wiederentdeckt-Dresdner-Malerinnen-Der-Romantik-Dresden-116639.html
Event_name                                                                                                             Wiederentdeckt! Dresdner Malerinnen der Romantik
                                                                                                    ...                                                   

In [164]:
flt = df["Event_URL"].str.contains('https://sachsen-net.com/veranstaltungen/heute/Von-Wanderlust-Und-Reisefrust', na=False)
id = max(df.index) + 1
print(id)
tmp = df[flt]
assert len(df[flt])>0 
df = df[~flt]
tmp = tmp.iloc[0]
print(tmp)
df.loc[id]=tmp
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-27"
df.loc[id, "extrCity"]="Dresden"
df.loc[id, "extrLocationDetails"]="01099 Dresden Nordstraße 28"
df.loc[id, "extrMultipleDaysDetails"] = "MI bis SO 12.00 bis 17.00 Uhr"

45203
Date_(GMT)                                                                                                                                          2024-09-11 13:35:10
Job_Link                              https://dashboard.browse.ai/teams/meslis/robots/7afcec8c-6cf5-48ab-8bfe-561611c64908/history/19403c5b-c176-4d67-bc6c-b92726ccf34c
Origin_URL                                                                                                               https://sachsen-net.com/veranstaltungen/heute/
Event_URL                                     https://sachsen-net.com/veranstaltungen/heute/Von-Wanderlust-Und-Reisefrust-Romantische-Natur-Und-Reisebilder-113200.html
Event_name                                                                           Von Wanderlust und Reisefrust. Romantische Natur- und Reisebilder im Dreiländereck
                                                                                                    ...                                                   

In [165]:
flt = df["Event_URL"].str.contains('https://sachsen-net.com/veranstaltungen/heute/Ausstellung-Home-Sweet-Home-Danny-Hermann-Radebeul-117962.html', na=False)
id = max(df.index) + 1
print(id)
tmp = df[flt]
assert len(df[flt])>0 
df = df[~flt]
tmp = tmp.iloc[0]
print(tmp)
df.loc[id]=tmp
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-07"
df.loc[id, "extrEndDate"] = "2024-10-31"
df.loc[id, "extrMultipleDaysDetails"] = "Samstag & Sonntag 14-18 Uhr"

45204
Date_(GMT)                                                                                                                                          2024-09-10 13:37:27
Job_Link                              https://dashboard.browse.ai/teams/meslis/robots/7afcec8c-6cf5-48ab-8bfe-561611c64908/history/0758270f-255b-4652-9301-72e97cb58f67
Origin_URL                                                                                                               https://sachsen-net.com/veranstaltungen/heute/
Event_URL                                                  https://sachsen-net.com/veranstaltungen/heute/Ausstellung-Home-Sweet-Home-Danny-Hermann-Radebeul-117962.html
Event_name                                                                                                                Ausstellung - Home Sweet Home - Danny Hermann
                                                                                                    ...                                                   

In [166]:
flt = df["Event_URL"].str.contains('https://sachsen-net.com/veranstaltungen/heute/Sonderausstellung-Andreas-Braeunsdorf-Malerei-Dresden-116638.html', na=False)
id = max(df.index) + 1
print(id)
tmp = df[flt]
assert len(df[flt])>0 
df = df[~flt]
tmp = tmp.iloc[0]
print(tmp)
df.loc[id]=tmp
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-07"
df.loc[id, "extrEndDate"] = "2024-09-29"

45205
Date_(GMT)                                                                                                                                          2024-09-12 13:36:10
Job_Link                              https://dashboard.browse.ai/teams/meslis/robots/7afcec8c-6cf5-48ab-8bfe-561611c64908/history/a0d0e5aa-5b94-421e-8003-303e051bba3c
Origin_URL                                                                                                               https://sachsen-net.com/veranstaltungen/heute/
Event_URL                                               https://sachsen-net.com/veranstaltungen/heute/Sonderausstellung-Andreas-Braeunsdorf-Malerei-Dresden-116638.html
Event_name                                                                                                             Sonderausstellung "Andreas Bräunsdorf - Malerei"
                                                                                                    ...                                                   

In [167]:
flt = df["Event_URL"].str.contains('https://sachsen-net.com/veranstaltungen/heute/Udo-Zimmermann-Ein-Moderner-Romantiker-Dresden-114359.html', na=False)
id = max(df.index) + 1
print(id)
tmp = df[flt]
assert len(df[flt])>0 
df = df[~flt]
tmp = tmp.iloc[0]
print(tmp)
df.loc[id]=tmp
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-09-29"
df.loc[id, "extrMultipleDaysDetails"] = "MI bis SO 12.00 bis 17.00 Uhr"

45206
Date_(GMT)                                                                                                                                          2024-09-12 13:36:10
Job_Link                              https://dashboard.browse.ai/teams/meslis/robots/7afcec8c-6cf5-48ab-8bfe-561611c64908/history/a0d0e5aa-5b94-421e-8003-303e051bba3c
Origin_URL                                                                                                               https://sachsen-net.com/veranstaltungen/heute/
Event_URL                                                      https://sachsen-net.com/veranstaltungen/heute/Udo-Zimmermann-Ein-Moderner-Romantiker-Dresden-114359.html
Event_name                                                                                                                    Udo Zimmermann – ein moderner Romantiker?
                                                                                                    ...                                                   

In [168]:
flt = df["Event_URL"].str.contains('https://sachsen-net.com/veranstaltungen/heute/Trickfilm-Plakat-Kunst-Arbeiten-Fuer-Den-Ddr-Filmverleih-113199.html', na=False)
id = max(df.index) + 1
print(id)
tmp = df[flt]
assert len(df[flt])>0 
df = df[~flt]
tmp = tmp.iloc[0]
print(tmp)
df.loc[id]=tmp
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-31"

45207
Date_(GMT)                                                                                                                                          2024-09-10 13:37:27
Job_Link                              https://dashboard.browse.ai/teams/meslis/robots/7afcec8c-6cf5-48ab-8bfe-561611c64908/history/0758270f-255b-4652-9301-72e97cb58f67
Origin_URL                                                                                                               https://sachsen-net.com/veranstaltungen/heute/
Event_URL                                            https://sachsen-net.com/veranstaltungen/heute/Trickfilm-Plakat-Kunst-Arbeiten-Fuer-Den-Ddr-Filmverleih-113199.html
Event_name                                                                                           TRICKFILM PLAKAT KUNST – Arbeiten für den DDR-Filmverleih Progress
                                                                                                    ...                                                   

In [169]:
flt = df["Event_URL"].str.contains('https://sachsen-net.com/veranstaltungen/heute/Panorama-Amazonien-Im-Panometer-Dresden-Dresden-113732.html', na=False)
id = max(df.index) + 1
print(id)
tmp = df[flt]
assert len(df[flt])>0 
df = df[~flt]
tmp = tmp.iloc[0]
print(tmp)
df.loc[id]=tmp
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-31"

45208
Date_(GMT)                                                                                                                                          2024-09-10 13:37:27
Job_Link                              https://dashboard.browse.ai/teams/meslis/robots/7afcec8c-6cf5-48ab-8bfe-561611c64908/history/0758270f-255b-4652-9301-72e97cb58f67
Origin_URL                                                                                                               https://sachsen-net.com/veranstaltungen/heute/
Event_URL                                                     https://sachsen-net.com/veranstaltungen/heute/Panorama-Amazonien-Im-Panometer-Dresden-Dresden-113732.html
Event_name                                                                                                                      Panorama AMAZONIEN im Panometer Dresden
                                                                                                    ...                                                   

In [170]:
flt = df["Event_URL"].str.contains('https://sachsen-net.com/veranstaltungen/heute/Zeitfenster-Spurensuche-Zur-Industriegeschichte-Im-Ernemannbau-Dresden-113361.html', na=False)
id = max(df.index) + 1
print(id)
tmp = df[flt]
assert len(df[flt])>0 
df = df[~flt]
tmp = tmp.iloc[0]
print(tmp)
df.loc[id]=tmp
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-20"

45209
Date_(GMT)                                                                                                                                          2024-09-10 13:37:27
Job_Link                              https://dashboard.browse.ai/teams/meslis/robots/7afcec8c-6cf5-48ab-8bfe-561611c64908/history/0758270f-255b-4652-9301-72e97cb58f67
Origin_URL                                                                                                               https://sachsen-net.com/veranstaltungen/heute/
Event_URL                              https://sachsen-net.com/veranstaltungen/heute/Zeitfenster-Spurensuche-Zur-Industriegeschichte-Im-Ernemannbau-Dresden-113361.html
Event_name                                                                                             „Zeitfenster“ Spurensuche zur Industriegeschichte im Ernemannbau
                                                                                                    ...                                                   

In [171]:
flt = df["Event_URL"].str.contains('https://sachsen-net.com/veranstaltungen/heute/Physics-Of-Life-Physik-Des-Lebens-Dresden-109648.html', na=False)
id = max(df.index) + 1
print(id)
tmp = df[flt]
assert len(df[flt])>0 
df = df[~flt]
tmp = tmp.iloc[0]
print(tmp)
df.loc[id]=tmp
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-27"

45210
Date_(GMT)                                                                                                                                          2024-09-10 13:37:27
Job_Link                              https://dashboard.browse.ai/teams/meslis/robots/7afcec8c-6cf5-48ab-8bfe-561611c64908/history/0758270f-255b-4652-9301-72e97cb58f67
Origin_URL                                                                                                               https://sachsen-net.com/veranstaltungen/heute/
Event_URL                                                           https://sachsen-net.com/veranstaltungen/heute/Physics-Of-Life-Physik-Des-Lebens-Dresden-109648.html
Event_name                                                                                                                          Physics of Life - Physik des Lebens
                                                                                                    ...                                                   

In [172]:
flt = df["Event_URL"].str.contains('https://sachsen-net.com/veranstaltungen/heute/Dresdner-Herbstvolksfest-Der-Freizeitpark-Dresden-119283.html', na=False)
id = max(df.index) + 1
print(id)
tmp = df[flt]
assert len(df[flt])>0 
df = df[~flt]
tmp = tmp.iloc[0]
print(tmp)
df.loc[id]=tmp
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-10-03"
df.loc[id, "extrEndDate"] = "2024-10-31"
df.loc[id, "extrMultipleDaysDetails"] = """ Montag bis Donnerstag 15:00 - 22:00 Uhr
Freitag 15:00 - 23:00 Uhr
Samstag 14:00 - 23:00 Uhr
Sonntag 14:00 - 22:00 Uhr
SCHLIESSTAGE 21.10., 22.10., 28.10., 29.10.2024"""
df.loc[id, "extrLocationName"]="Volksfestgelände Pieschener Allee"
df.loc[id, "extrLocationDetails"]="01067 Dresden Pieschener Allee 1"

45211
Date_(GMT)                                                                                                                                          2024-10-05 13:36:41
Job_Link                              https://dashboard.browse.ai/teams/meslis/robots/7afcec8c-6cf5-48ab-8bfe-561611c64908/history/675f52d6-1316-4d94-91a2-669b20e69fd3
Origin_URL                                                                                                               https://sachsen-net.com/veranstaltungen/heute/
Event_URL                                                   https://sachsen-net.com/veranstaltungen/heute/Dresdner-Herbstvolksfest-Der-Freizeitpark-Dresden-119283.html
Event_name                                                                                                                  Dresdner Herbstvolksfest - Der Freizeitpark
                                                                                                    ...                                                   

In [173]:
len(df)

490

In [87]:
df.loc[:, "count_Event_URL"]= df.Event_URL.map(df.Event_URL.value_counts())

# bookmark

In [90]:
df.query("count_Event_URL>10")[["isOnMultipleDays", "Event_URL"]].value_counts()

isOnMultipleDays  Event_URL                                                                                                                         
True              https://sachsen-net.com/veranstaltungen/heute/Dresdner-Herbstmarkt-Dresden-118830.html                                                23
False             https://sachsen-net.com/veranstaltungen/heute/Erlebnisausstellung-Die-Spiele-Des-Koenigs-Mit-Ringrennkarussell-Dresden-116914.html    19
                  https://sachsen-net.com/veranstaltungen/heute/Palais-Zeitspruenge-Xvi-Kostuemausstellung-2024-Dresden-118837.html                     17
                  https://sachsen-net.com/veranstaltungen/heute/Portraits-Hellerau-Photography-Award-2024-Dresden-118206.html                           17
True              https://sachsen-net.com/veranstaltungen/heute/Portraits-Hellerau-Photography-Award-2024-Dresden-118206.html                           17
                  https://sachsen-net.com/veranstaltungen/heute/Escape-Room-

In [ ]:

df_urls = df.query("count_Event_URL>1 ")[required_cols_except_start_date].drop_duplicates()
print(len(df_urls))
df_urls.sort_values("Event_URL")

71


,Event_URL,count_Event_URL,isOnMultipleDays,extrStartTime,extrEndTime,extrEndDate,extrMultipleDaysDetails,extrCity,extrStreetHouseNumber,extrLocationDetails,extrLocationName
9865,https://sachsen-net.com/veranstaltungen/heute/33-Ausstellung-Historische-Fahrraeder-Im-Dresdner-Stallhof-Dresden-118975.html,2.0,False,NaN,NaN,NaN,NaN,Dresden,NaN,NaN,Stallhof Dresden
13859,https://sachsen-net.com/veranstaltungen/heute/Ausstellung-Floristische-Meisterstuecke-In-Der-Orangerie-Dresden-Pillnitz-118991.html,2.0,True,NaN,NaN,2023-09-29,NaN,Dresden,NaN,NaN,Schlosspark Pillnitz
14265,https://sachsen-net.com/veranstaltungen/heute/Ausstellung-Floristische-Meisterstuecke-In-Der-Orangerie-Dresden-Pillnitz-118991.html,2.0,False,NaN,NaN,NaN,NaN,Dresden,NaN,NaN,Schlosspark Pillnitz
18048,https://sachsen-net.com/veranstaltungen/heute/Die-Hofewiesn-Das-Familienoktoberfest-In-Der-Hofewiese-Dresden-118977.html,4.0,True,NaN,NaN,NaN,Am gesamten langen Wochenende rund um den Nationalfeiertag (Details unspecified),Dresden,NaN,NaN,Hofewiese
17412,https://sachsen-net.com/veranstaltungen/heute/Die-Hofewiesn-Das-Familienoktoberfest-In-Der-Hofewiese-Dresden-118977.html,4.0,True,NaN,NaN,2024-10-03,NaN,Dresden,NaN,NaN,Hofewiese
...,...,...,...,...,...,...,...,...,...,...,...
17423,https://sachsen-net.com/veranstaltungen/heute/Streetfood-On-Tour-In-Dresden-Dresden-119282.html,2.0,True,NaN,NaN,NaN,"Wed: 14:00-22:00, Thu-Sat: 11:00-22:00, Sun: 11:00-20:00",Dresden,NaN,NaN,Prager Straße
18059,https://sachsen-net.com/veranstaltungen/heute/Streetfood-On-Tour-In-Dresden-Dresden-119282.html,2.0,False,NaN,NaN,NaN,NaN,Dresden,NaN,NaN,Prager Straße
28255,https://sachsen-net.com/veranstaltungen/heute/Verlauf-Stefanie-Hollerbach-Trifft-Staendige-Ausstellung-Dresden-118889.html,7.0,True,NaN,NaN,NaN,"Tue-Sun: 10:00-18:00, Fri: 10:00-19:00",Dresden,NaN,NaN,Städtische Galerie Dresden
29097,https://sachsen-net.com/veranstaltungen/heute/Verlauf-Stefanie-Hollerbach-Trifft-Staendige-Ausstellung-Dresden-118889.html,7.0,False,NaN,NaN,NaN,NaN,Dresden,NaN,NaN,Städtische Galerie Dresden


In [117]:
len(df)

467

In [118]:
# df_urls.to_csv(f"{DATA_FOLDER}urls_to_check.csv", index=False)

In [107]:
df_scraped = pd.read_csv(f"{DATA_FOLDER}urls_to_check_scraped.csv")

In [109]:
df_scraped.head()

,Extract Date,Status,Task Link,Origin URL,dates,location_name,location_details
0,2024-12-20 07:15:38,Successful,https://dashboard.browse.ai/workspaces/meslis/robots/67103f61-157e-48d6-888d-fd67a8e4e6ee/history/2ffadbb3-8673-4003-a7cc-0a13e07416aa,https://sachsen-net.com/veranstaltungen/heute/Eislaufsaison-Der-Joynext-Arena-Dresden-119538.html,26.10.2024 bis 02.03.2025 - JOYNEXT Arena in Dresden,JOYNEXT Arena,01067 Dresden Magdeburger Str. 10
1,2024-12-20 07:15:38,Successful,https://dashboard.browse.ai/workspaces/meslis/robots/67103f61-157e-48d6-888d-fd67a8e4e6ee/history/704a0dad-4ee0-4728-830e-03c109ff3dfa,https://sachsen-net.com/veranstaltungen/heute/Verlauf-Stefanie-Hollerbach-Trifft-Staendige-Ausstellung-Dresden-118889.html,25.10.2024 bis 25.05.2025 - Städtische Galerie Dresden in Dresden,Städtische Galerie Dresden,01067 Dresden Wilsdruffer Straße 2
2,2024-12-20 07:15:34,Successful,https://dashboard.browse.ai/workspaces/meslis/robots/67103f61-157e-48d6-888d-fd67a8e4e6ee/history/0dd75648-aa1d-4018-b12f-85133a32b03f,https://sachsen-net.com/veranstaltungen/heute/Echtzeit-Neue-Werke-Fuer-Die-Kunstsammlungen-Der-Stadt-118888.html,25.10.2024 bis 21.04.2025 - Städtische Galerie Dresden in Dresden,Städtische Galerie Dresden,01067 Dresden Wilsdruffer Straße 2
3,2024-12-20 07:15:34,Successful,https://dashboard.browse.ai/workspaces/meslis/robots/67103f61-157e-48d6-888d-fd67a8e4e6ee/history/7261d5b7-23f7-4e3b-8fe6-dc3b14247d7c,https://sachsen-net.com/veranstaltungen/heute/Parkseminar-Im-Grossen-Garten-Dresden-2024-Dresden-118838.html,25.10.2024 bis 26.10.2024 - Palais im Großen Garten in Dresden,Palais im Großen Garten,01219 Dresden Hauptallee
4,2024-12-20 07:15:34,Successful,https://dashboard.browse.ai/workspaces/meslis/robots/67103f61-157e-48d6-888d-fd67a8e4e6ee/history/502b60d3-deb8-4cde-9cbc-446ae636d1fe,https://sachsen-net.com/veranstaltungen/heute/Sonderausstellung-Wilhelm-Mueller-Dresden-118890.html,19.10.2024 bis 23.02.2025 - Leonhardi-Museum in Dresden,Leonhardi-Museum,01326 Dresden Grundstraße 26


In [ ]:
for row, index in df_scraped.iterrows():
    url = row["Origin URL"]
    
    

In [115]:
len(df)

674

In [507]:
flt = (df.extrStartTime=="nan") & (df.isOnMultipleDays==False)
#df.loc[flt]

In [ ]:
df.query("isOnMultipleDays==False & OpenAIResponse_MultipleDaysDetails!='nan'")

In [ ]:
len(df[df.OpenAIResponse_StartDate.str.contains("2023") & (df.isOnMultipleDays==False)])

# final checks and keep only suitable observations

In [ ]:
len(df)

In [511]:
extracted_columns = []
for col in df.columns:
    if col.startswith("extr"):
        extracted_columns.append(col)
    
extracted_columns.append("isOnMultipleDays")

In [ ]:
for col in extracted_columns:
    print(f"{col=} has missing values:")
    flt = df[col]=="nan"
    print(len(df[flt]))


In [ ]:
flt = (df.isOnMultipleDays==False ) & (df.extrStartDate!="nan") & (df.extrStartTime!="nan") &\
    ( (df.extrLocationName!="nan") | (df.extrLocationDetails!="nan") | (df.extrStreetHouseNumber!="nan")) 
len(df[flt])

In [ ]:
df.loc[flt, extracted_columns]

## save

In [515]:
exclude_columns= ['Date (GMT)', 'Job Link']

subset_columns = [col for col in df.columns if col not in exclude_columns]

df = df.drop_duplicates(subset=subset_columns)

In [ ]:
len(df)

In [517]:
df = df.dropna(how="all", axis=1)

In [ ]:
# df.to_excel(f"{DATA_FOLDER}df_browse_ai {datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.xlsx")

In [521]:
df.to_csv(f"{DATA_FOLDER}df_browse_ai {datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.csv")

In [522]:
df.to_pickle(f"{DATA_FOLDER}df_browse_ai {datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.pkl")